# preprocessing

In [ ]:
library(datasets)
library(NbClust)
library(factoextra)
library(tidyverse)
library(janitor)
library(readxl) # read xls files
library(NbClust) # find
library(knitr) # find best no cluster
library(factoextra)
library(cluster)
library("factoextra")
library(caret)

In [ ]:
head(iris)

In [ ]:
summary(iris)

In [ ]:
boxplot(iris)

## remove outliares

### IQR

In [ ]:
#find Q1, Q3, and interquartile range for values in column A
Q1 <- quantile(iris$Sepal.Width, .25)
Q3 <- quantile(iris$Sepal.Width, .75)
IQR <- IQR(iris$Sepal.Width)

#only keep rows in dataframe that have values within 1.5*IQR of Q1 and Q3
no_outliers <- subset(iris, iris$Sepal.Width> (Q1 - 1.5*IQR) & iris$Sepal.Width< (Q3 + 1.5*IQR))

#view row and column count of new data frame
dim(no_outliers)

In [ ]:
boxplot(no_outliers)

## Normalization

In [ ]:
hist(iris$Sepal.Width)

In [ ]:
hist(iris$Petal.Length)

In [ ]:
z = function(x) {
  return((x - mean(x)) / sd(x))
}

In [ ]:
dfNorm <- as.data.frame(lapply(no_outliers[1:4], z_score))

In [ ]:
hist(dfNorm$Sepal.Width)

In [ ]:
hist(dfNorm$Petal.Length)

In [ ]:
summary(dfNorm)

In [ ]:
head(dfNorm)

In [ ]:
dfNorm$Species <- no_outliers$Species

In [ ]:
head(dfNorm)

Clustering automated tools

### nb

In [ ]:
set.seed(26)
clusterNo=NbClust(dfNorm[1:4],distance="euclidean", min.nc=2,max.nc=10,method="kmeans",index="all")

## elbow

In [ ]:
k = 2:10
set.seed(42)	
WSS = sapply(k, function(k) {kmeans(dfNorm[1:4], centers=k)$tot.withinss})

In [ ]:
plot(k, WSS, type="l", xlab= "Number of k", ylab="Within sum of squares")

In [ ]:
# Silhoutte method
fviz_nbclust(dfNorm[1:4], kmeans, method = "silhouette") +
 labs(subtitle = "Silhouette method")

## kmeans

In [ ]:
x = dfNorm[1:4]
y=dfNorm$Species 

In [ ]:
kc <- kmeans(x,4)

In [ ]:
kc

In [ ]:
table(y,kc$cluster)

In [ ]:
fviz_cluster(kc,data=dfNorm[1:4])

In [ ]:
# implement the metrix
confusionMatrix(

 factor(kc$cluster, levels = 1:4),

 factor(as.numeric(dfNorm$Species), levels = 1:4)
)

# PCA

#PCA

In [ ]:
head(dfNorm)

In [ ]:
myPr <- prcomp(dfNorm[1:4])

In [ ]:
# myPr <- prcomp(iris[, -5], scale = TRUE)# 

In [ ]:
# plot((iris$Sepal.Length - mean(iris$Sepal.Length)) / sd(iris$Sepal.Length))

In [ ]:
myPr

In [ ]:
summary(myPr)

In [ ]:
plot(myPr, type = "l")

In [ ]:
biplot(myPr)

In [ ]:
biplot(myPr, scale = 0)

In [ ]:
str(myPr)

In [ ]:
myPr$x

In [ ]:
iris2 <- cbind(dfNorm, myPr$x)

In [ ]:
head(iris2)

In [ ]:
library(ggplot2)

ggplot(iris2, aes(PC3,PC4,col = Species, fill = Species)) +
  stat_ellipse(geom = "polygon", col = "black", alpha = 0.5) +
  geom_point(shape = 21, col = "black")

In [ ]:
cor(iris[, -5], iris2[, 6:9])

In [ ]:
df_last <- iris2[8:9]

In [ ]:
df_last$Species = iris2$Species

In [ ]:
head(df_last)

In [ ]:
x = df_last[1:2]
y=df_last$Species 

In [ ]:
kc <- kmeans(x,4)

In [ ]:
kc

In [ ]:
fviz_cluster(kc,data=df_last[1:2])